# RISC-V Business

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Ramshreyas/Ethereum-Articles/blob/main/2/RISC-V_Business.ipynb)

!["RISC-V Business"](https://szmer.info/pictrs/image/6313c4e4-5b15-41ee-afa8-a7765cda4893.png)

Angelina Jolie in Hackers, 1995

It's almost 2025, and the great RISC revolution that has been looming for decades is finally threatening to materialize. The Ethereum Foundation is [funding a formal verification effort for RISC-V ZKVMs](https://verified-zkevm.org/), chip industry legends like [Jim Keller](https://en.wikipedia.org/wiki/Jim_Keller_(engineer)) are starting RISC-V-driven technology efforts like [tenstorrent](https://tenstorrent.com/en/vision/tenstorrent-risc-v-and-chiplet-technology-selected-to-build-the-future-of-ai-in-japan), RISC-V International (the governing body for the RISC-V standard) has [ratified the RISC-V specifications](https://riscv.org/announcements/2024/04/risc-v-international-achieves-milestone-with-ratification-of-40-specifications-in-two-years/), laying the ground for an open standard that industry can now build upon, and Justin Drake in his ambitious [Beam Chain proposal](https://youtu.be/lRqnFrqpq4k) outlined how the onset of RISC-V can massively accelerate Ethereum's ponderous roadmap dramatically on many fronts.

This article aims to introduce RISC-V, motivate its importance to the Ethereum vision, and, with a few toy code examples, connect the dots across the entire tech stack from high level code down to the metal.

---

### 1. ZK is Ethereum's future

![The Verge](https://ramshreyas.github.io/Ethereum-Articles/assets/imgs/TheVerge.png)

[Possible futures of the Ethereum protocol, part 4: The Verge](https://vitalik.eth.limo/general/2024/10/23/futures4.html)

As Vitalik outlines in his blog post about [The Verge](https://vitalik.eth.limo/general/2024/10/23/futures4.html): 

> "One of the most powerful things about a blockchain is the fact that anyone can run a node on their computer and verify that the chain is correct. Even if 95% of the nodes running the chain consensus (PoW, PoS...) all immediately agreed to change the rules, and started producing blocks according to the new rules, everyone running a fully-verifying node would refuse to accept the chain... However, for this property to hold, running a fully-verifying node needs to be actually feasible for a critical mass of people... Today, running a node is possible on a consumer laptop (including the one being used to write this post), but doing so is difficult. The Verge is about changing this, and making fully-verifying the chain so computationally affordable that every mobile wallet, browser wallet, and even smart watch is doing it by default."

The goal of 'The Verge' is to make verifying blocks as easy as verifying a SNARK/STARK proof. Achieving this would mean the complexity of running a validator is reduced to just being able to download some data and verifying a 'ZK' proof. This reduces the barrier to entry for becoming a validator (along with proposals [to reduce the minimum staking amount]), and increases Ethereum's censorship resisitance and resilience.

*Note: 'ZK' has unfortunately become a catch-all term which is not always accurate - we will address this in the next section, but keep using it for now.*

But what does it take to verify such a proof, and, indeed, produce it in the fist place?

To understand this, let us first understand that we would be proving three kinds of things in each block:

1. [State](https://vitalik.eth.limo/general/2024/10/23/futures4.html#1): What are all the balances, addresses and raw data currently, and do they agree with the entire history of the chain? 
2. [Execution](https://vitalik.eth.limo/general/2024/10/23/futures4.html#2): Now that we know what the current state is, what is the post execution state transition, once all the transactions in the current block are executed? Has this been done correctly without any cheating?
3. [Consensus](https://vitalik.eth.limo/general/2024/10/23/futures4.html#3): Has the cryptoeconomics of the proof of stake mechanism (which ultimately secures the chain) been executed correctly? Have all the validator actions, balances, withdrawals signatures etc been updated correctly?

>"Stateless verification is a technology that allows nodes to verify blocks without having the entire state. Instead, each block comes with a witness, which includes (i) the values (eg. code, balances, storage) at the specific locations in the state that the block will access, and (ii) a cryptographic proof that those values are correct. "

State in Ethereum is currently stored in a (Merkle Patricia) tree structure, and is currently "extremely unfriendly to implementing any cryptographic proof scheme" per Vitalik. This is one of the motivations to move to a more friendly structure such as Verkle or STARKed binary trees. Changing this structure has already become a priority expressly to enable SNARK/STARK proving of state as described in the quote above.

>"...you should be able to verify an Ethereum block by (i) downloading the block, or perhaps even only small parts of the block with data availability sampling, and (ii) verifying a small proof that the block is valid... Getting to this point requires having SNARK or STARK proofs of (i) the consensus layer (ie. the proof of stake), and (ii) the execution layer (ie. the EVM)... Today, validity proofs for the EVM are inadequate in two dimensions: security and prover time."

The time taken to prove a block with sufficiently 'weak' hardware is an essential component of The Verge - software run by solo home stakers, or even users with a smartwatch for that matter, needs to be sufficiently fast enough so that an "Ethereum block can be proven in less than ~4 seconds.".

>"If we want it to be possible to fully verify an Ethereum block with a SNARK, then the EVM execution is not the only part we need to prove. We also need to prove the consensus: the part of the system that handles deposits, withdrawals, signatures, validator balance updates, and other elements of the proof-of-stake part of Ethereum."

While this is a significantly simpler problem than full zkEVM proving, it will need to be done 'from scratch' as these efforts are not also being driven by the Layer 2s, who are also heavily investing developing zkEVMs for their own purposes. 

The main takeaway is this: the future of Ethereum is ZK (or PM, really, as we will introduce in the next section) - and there are significant hurdles to cross, especially in terms of performance, before we can get there.

---

### 2. Programmable Cryptography and the rise of 'Glue and Co-processor' architectures

**2.1 Programmable Cryptography**

As introduced in a series of posts on [Programmable cryptography](https://0xparc.org/blog/programmable-cryptography-1) by [gubsheep](https://x.com/gubsheep):

>"Cryptography is undergoing a generational transition, from special-purpose cryptography to programmable cryptography."

Currently, the majority of cryptography use-cases are special-purpose - custom protocols which are designed to do one thing, and one thing only. Public key encryption and the signature schemes that underlie WWW certification are obvious examples. The next generation of 'Programmable Cryptography' introduces the notion of cryptographic primitives that are flexible and composable - "they allow us to perform general-purpose computation inside or on top of cryptographic protocols" per Gubsheep.

This means we now have the capacity to perform any computation our little hearts may desire, and get cryptographic verification that it was performed correctly, without having to eat glass and develop specialized cryptographic proofs for a multitude of critical functions scattered across the code. This is how we can get a ZK proof of valide block execution by using EVM complient code such as reth to execute the block within the Risc0 zkVM (VM, not EVM). This is a rather astounding claim - all EVM protocols that were interacted with in that particular block can get ZK proofs of valid execution, seemingly for free - without changing anything about their own code and writing any specialized code. It is handled entirely by a general purpose zkVM with programmable cryptography primitives. 

So Programmable Cryptography gives us zkVMs, which in turn gives us:

**2.2 zkEVMs**

In August 2023, [Risc0 released Zeth, a ZK execution layer block prover for Ethereum and Optimism](https://github.com/risc0/zeth). Since then, multiple teams have followed suit, such as [Succinct Labs' SP1](https://github.com/succinctlabs/sp1). By running a compatible execution client like [Reth](https://github.com/paradigmxyz/reth) inside a zkVM like Zeth, we can obtain a ZK proof of valid block execution including:

- Verifying transaction signatures.
- Verifying account & storage state against the parent block’s state root.
- Applying transactions.
- Paying the block reward.
- Updating the state root.

There are actually multiple types of zkEVMs, occupying different parts of the trade=off space between Ethereum compatibility and proving speed:

- Type 1: Fully Ethereum-compatible, inheriting all of Ethereum's infrastructure and security, but with longer prover times (the time it takes to generate zk-proofs).
- Type 2: Fully EVM-equivalent, striking a balance between compatibility and prover times.
- Type 2.5: EVM-equivalent except for gas costs, prioritizing scalability but with some compatibility limitations for certain dApps and tools.
- Type 3: Almost EVM-equivalent, making it easier to build and offering better scalability, but requiring some modifications to Ethereum dApps.
- Type 4: Compiles smart contract source code directly to ZK-SNARKs, achieving the fastest prover times but with the lowest compatibility with Ethereum dApps and tools. This variety of zkEVM types allows developers to choose the solution that best suits their specific needs and priorities.

[Since RiscZero’s pioneering initial release of their VM, general-purpose ZKVMs based on RISC-V have surged in popularity.](https://argument.xyz/blog/riscv-good-bad/)

But zkEVMs did not ned to use RISC-V. In fact, many of the early pioneer zkEVMs such as Polygon zero, ZKSync, scroll and others targeted various architectures in their own PC adventures. How does RISC-V enter the picture?

**2.3 The rise and rise of Glue and Co-processor architectures**



---

### RISC-V Features That Empower Ethereum Development

1. Open and Modular Design
    How RISC-V allows Ethereum teams to design custom hardware (e.g., ZKP accelerators).
2. Auditable and Trustless Hardware
    Aligning RISC-V’s transparency with Ethereum’s trustless ethos.
3. Portability Across Architectures
    Cross-platform compatibility for Ethereum clients and cryptographic tooling.

---

### Lightweight Code Examples: RISC-V in Action

1. A Simple ZKP Operation: Modular Arithmetic
    Code example: A RISC-V instruction for modular addition.
2. Optimizing Keccak on RISC-V
    Code snippet: A hashing operation in RISC-V.
3. FFT Steps in zk-SNARK Provers
    Code walkthrough: Using RISC-V to perform polynomial evaluations.

---

### How RISC-V Works: ISA Basics for Ethereum Developers

1. The ISA as the Software-Hardware Contract
    Explanation of ISAs and how they enable software portability.
2. Stack-Based Machines vs. Register-Based Architectures
    Contrasting the EVM with RISC-V for Ethereum developers.
3. RISC-V’s Modular Extensions
    How Ethereum-specific operations can be implemented efficiently.

---


### Why RISC-V is Uniquely Suited for Ethereum

1. Open Collaboration in Blockchain and Hardware
    Community-driven innovation in Ethereum and RISC-V.
2. Future-Proofing Ethereum with RISC-V
    RISC-V’s adaptability for evolving zk and cryptographic needs.
3. A Decentralized Approach to Hardware Design
    Enabling hardware decentralization with open-source RISC-V.

---

### Conclusion

1. Call to Action for Ethereum Developers
    Start experimenting with RISC-V and contribute to its ecosystem.
2. The Path Forward for RISC-V and Ethereum
    Collaboration opportunities and the importance of open hardware for the future of decentralized technologies.

---